# King Khalid International Airport (RUH) - Flights Analysis

**Goal:** Explore flight movements data for RUH to understand traffic patterns (time, airlines, and flight types) and produce a clean dataset that can be used in a dashboard.

**Tools:** PySpark (Spark DataFrame API)

### 1) Environment setup

In [ ]:
#Start Spark
!pip -q install pyspark

from pyspark.sql import SparkSession, functions as F

spark = (
    SparkSession.builder
    .appName("RUH Airport Ops Analytics")
    .getOrCreate()
)

### 2) Load the dataset

In [ ]:
df = spark.read.parquet("flights_RUH.parquet")

In [ ]:
df.show(5)

+-------------+---------------+------------+--------------+-----------------+------------+------------+-------+-----------+---------------+-------+--------+-------------------+-------------------+-------------------+-----------------+----------------+------------------------+------------------------+------------------------+-------------------------+--------------------------+----------------------------+
|flight_number| aircraft.model|aircraft.reg|aircraft.modeS|     airline.name|airline.iata|airline.icao| status|flight_type|codeshareStatus|isCargo|callSign|origin_airport_name|origin_airport_icao|origin_airport_iata|movement.terminal|movement.quality|destination_airport_icao|destination_airport_iata|destination_airport_name|movement.airport.timeZone|movement.scheduledTime.utc|movement.scheduledTime.local|
+-------------+---------------+------------+--------------+-----------------+------------+------------+-------+-----------+---------------+-------+--------+-------------------+------

### 3) Data cleaning and feature engineering

In [ ]:
from pyspark.sql import functions as F

df_clean = (
    df
    # Parse local time with timezone offset (+03:00)
    .withColumn(
        "scheduled_local_utc",
        F.to_timestamp(F.col("`movement.scheduledTime.local`"), "yyyy-MM-dd HH:mmXXX")
    )
    # Convert UTC timestamp to Riyadh local time for correct hourly/daily analysis
    .withColumn(
        "scheduled_local",
        F.from_utc_timestamp("scheduled_local_utc", "Asia/Riyadh")
    )

    # Parse UTC scheduled time
    .withColumn(
        "scheduled_utc",
        F.to_timestamp(F.col("`movement.scheduledTime.utc`"), "yyyy-MM-dd HH:mm'Z'")
    )

    # Build time-based features using Riyadh local time
    .withColumn("date",  F.to_date("scheduled_local"))
    .withColumn("hour",  F.hour("scheduled_local"))
    .withColumn("month", F.date_format("scheduled_local", "yyyy-MM"))

    # Other features
    .withColumn("airline",   F.coalesce(F.col("`airline.name`"), F.lit("Unknown")))
    .withColumn("dest_icao", F.coalesce(F.col("destination_airport_icao"), F.lit("UNK")))
    .withColumn("is_domestic", F.col("dest_icao").startswith("OE"))
)

In [ ]:
df_clean.select("`movement.scheduledTime.local`", "scheduled_local").show(5, truncate=False)

+----------------------------+-------------------+
|movement.scheduledTime.local|scheduled_local    |
+----------------------------+-------------------+
|2025-03-15 00:01+03:00      |2025-03-15 00:01:00|
|2025-03-15 00:05+03:00      |2025-03-15 00:05:00|
|2025-03-15 00:05+03:00      |2025-03-15 00:05:00|
|2025-03-15 00:10+03:00      |2025-03-15 00:10:00|
|2025-03-15 00:15+03:00      |2025-03-15 00:15:00|
+----------------------------+-------------------+
only showing top 5 rows


### 4) Exploratory analysis (KPIs)

In [ ]:
# 3) KPI Tables
# 3.1 Total flights
total = df_clean.count()
print("Total flights:", total)

Total flights: 153308


In [ ]:
# 3.2 Arrivals vs Departures based on flight_type
flight_type_kpi = (
    df_clean.groupBy("flight_type")
    .count()
    .withColumn("pct", F.round(F.col("count") / F.lit(total) * 100, 2))
    .orderBy(F.desc("count"))
)
flight_type_kpi.show(truncate=False)

+-----------+-----+-----+
|flight_type|count|pct  |
+-----------+-----+-----+
|departure  |78252|51.04|
|arrival    |75056|48.96|
+-----------+-----+-----+



In [ ]:
# 3.3 Peak Operating Hours
hourly_kpi = df_clean.groupBy("hour") \
    .count() \
    .orderBy(F.desc("count")) \
    .show()

+----+-----+
|hour|count|
+----+-----+
|  18| 8142|
|  20| 8131|
|  16| 7904|
|  19| 7545|
|  22| 7516|
|  21| 7479|
|  23| 7477|
|  17| 7419|
|  15| 7356|
|   9| 6811|
|  10| 6669|
|  14| 6579|
|  11| 6472|
|   8| 6282|
|  13| 6049|
|   6| 5964|
|   7| 5761|
|  12| 5651|
|   0| 5600|
|   2| 4993|
+----+-----+
only showing top 20 rows


In [ ]:
# 3.4 Top 10 Airlines by Flight Volume
top_airlines = df_clean.groupBy("airline") \
    .count() \
    .orderBy(F.desc("count")) \
    .show(10, truncate=False)

+-------------+-----+
|airline      |count|
+-------------+-----+
|Saudi Arabian|60886|
|flynas       |33935|
|flyadeal     |24835|
|Gulf Air     |2198 |
|flydubai     |2059 |
|EgyptAir     |1878 |
|Qatar        |1803 |
|Etihad       |1539 |
|Emirates     |1365 |
|Turkish      |1260 |
+-------------+-----+
only showing top 10 rows


In [ ]:
# 3.5 Monthly traffic trend
monthly_kpi = df_clean.groupBy("month") \
    .count() \
    .orderBy("month") \
    .show(50, truncate=False)

+-------+-----+
|month  |count|
+-------+-----+
|2025-03|10443|
|2025-04|21280|
|2025-05|22077|
|2025-06|21929|
|2025-07|23734|
|2025-08|24042|
|2025-09|22751|
|2025-10|7052 |
+-------+-----+



In [ ]:
# 3.6 Domestic vs International Traffic Split
dom_int_kpi = (
    df_clean.groupBy("is_domestic")
    .count()
    .withColumn("pct", F.round(F.col("count") / F.lit(total) * 100, 2))
    .orderBy(F.desc("count"))
)
dom_int_kpi.show(truncate=False)

+-----------+-----+----+
|is_domestic|count|pct |
+-----------+-----+----+
|true       |79413|51.8|
|false      |73895|48.2|
+-----------+-----+----+



In [ ]:
final_dataset = df_clean.select(
    "flight_type",
    "airline",
    "is_domestic",
    "date",
    "month",
    "hour"
)

final_dataset.show(5)

+-----------+-----------------+-----------+----------+-------+----+
|flight_type|          airline|is_domestic|      date|  month|hour|
+-----------+-----------------+-----------+----------+-------+----+
|  departure|         Air Sial|      false|2025-03-15|2025-03|   0|
|  departure|           flynas|      false|2025-03-15|2025-03|   0|
|  departure|Starlight Airline|      false|2025-03-15|2025-03|   0|
|  departure|         flyadeal|       true|2025-03-15|2025-03|   0|
|  departure|              KLM|       true|2025-03-15|2025-03|   0|
+-----------+-----------------+-----------+----------+-------+----+
only showing top 5 rows


### 5) Export the data as csv file for dashboarding using Power BI

In [ ]:
final_dataset.coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("RUH_final_dataset")

### Key Insights

* Operations are balanced
(51% departures, 49% arrivals).

* Highest activity occurs between 16:00–23:00, with a peak at 18:00, indicating evening congestion pressure.

* Saudi Arabian Airlines, flynas, and flyadeal account for the majority of movements.

* Traffic peaks in July–August, with domestic flights slightly higher than international.

### Conclusion

RUH shows balanced operations, strong evening peaks, and clear summer seasonality, with domestic airlines driving most traffic. These patterns highlight the need for focused resource planning during peak hours and high-demand months.